# Bose-Hubbard Model
*****
The Bose-Hubbard Hamiltonian is given by: $$\hat{H}=-J\sum_{\langle i,j\rangle}\left(\hat{b}_{i}^\dagger\hat{b}_{j}+\hat{b}_{i}\hat{b}_{j}^\dagger\right)+\frac{U}{2}\sum_{i} \hat{n}_i\left(\hat{n}_i-\hat{I}\right)+V\sum_{\langle i,j\rangle}\hat{n}_i\hat{n}_j-\mu\sum_{i}\hat{n}_i \$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

Suppose we have two wells and one total particle in the system. Try computing the Hamiltonian for this system with $V = 0$.

In [2]:
basis1 = basis(2,0)
basis2 = basis(2,1)
def number(N):
    return(create(N)*destroy(N))
number(3)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]

In [16]:
def NearestNeighbor(Sites):
    l=[]
    for i in range(Sites):
        for k in range(2):
            l1=[]
            l1.append(i)
            l1.append((i-1)+2*k)
            l.append(l1)
    l.remove(l[2*Sites-1])
    l.remove(l[0])
    return(l)

NearestNeighbor(5)

[[0, 1], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3]]

In [157]:
def ldestroy(N,Sites,Particles):
    l=[]
    for i in range(N):
        l.append(identity(Particles+1))
    l.append(destroy(Particles+1))
    for i in range(Sites-N-1):
        l.append(identity(Particles+1))
    if len(l) == 1:
        s=l[0]
    elif len(l) > 1:
        s=l[0]
        a=0
        ints=[]
        for i in range(len(l)-1):
            s=tensor(s,l[i+1])
            ints.append(s)
    return(s)
    
ldestroy(0,1,3)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.         1.         0.         0.        ]
 [0.         0.         1.41421356 0.        ]
 [0.         0.         0.         1.73205081]
 [0.         0.         0.         0.        ]]

In [153]:
for i in range(0):
    print('Hello')

In [5]:
def lcreate(N,Sites,Particles):
    return(ldestroy(N,Sites,Particles).dag())

In [6]:
fock_dm(4,2).dims[0][0]1

4

In [139]:
def BHHamOLD(Sites,Particles,J=1,U=2,V=0,mu=1):
    def a(Location):
        return(ldestroy(Location,Sites,Particles))
    def adag(Location):
        return(lcreate(Location,Sites,Particles))
    def n(Location):
        return(lcreate(Location,Sites,Particles)*ldestroy(Location,Sites,Particles))
    sum1=0
    sum2=0
    sum3=0
    sum4=0
    NN = NearestNeighbor(Sites)
    for i in range(len(NN)):
        sum1 += adag(NN[i][0])*a(NN[i][1]) + a(NN[i][0])*adag(NN[i][1])
    for i in range(Sites):
        sum2 += Qobj(n(i).data*(n(i).data-identity(n(i).shape[0]).data))
    for i in range(len(NN)):
        sum3 += n(NN[i][0])*n(NN[i][1])
    for i in range(Sites):
        sum4 += n(i)
    H = Qobj(-J/2*sum1.data + U/2*sum2.data + V/2*sum3.data - mu*sum4.data)
    return(H)

BHHamOLD(2,1,1,0,0,1)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.]
 [ 0. -1. -1.  0.]
 [ 0. -1. -1.  0.]
 [ 0.  0.  0. -2.]]

In [164]:
def BHHam(Sites,Particles,J=1,U=2,V=0,mu=1):
    def a(Location):
        return(ldestroy(Location,Sites,Particles))
    def adag(Location):
        return(lcreate(Location,Sites,Particles))
    def n(Location):
        return(lcreate(Location,Sites,Particles)*ldestroy(Location,Sites,Particles))
    sum1=0
    sum2=0
    sum3=0
    sum4=0
    for i in range(Sites-1):
        sum1 += adag(i)*a(i+1) + a(i)*adag(i+1)
    for i in range(Sites):
        sum2 += Qobj(n(i).data*(n(i).data-identity(n(i).shape[0]).data))
    for i in range(Sites-1):
        sum3 += n(i)*n(i+1)
    for i in range(Sites):
        sum4 += n(i)
    if sum1 == 0:
        sum1 = Qobj(np.zeros(sum2.shape))
    if sum3 == 0:
        sum3 = Qobj(np.zeros(sum2.shape))
    H = Qobj(-J*sum1.data + U/2*sum2.data + V*sum3.data - mu*sum4.data)
    return(H)

BHHam(1,3,1,0,0,1)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -2.  0.]
 [ 0.  0.  0. -3.]]

In [172]:
H = BHHam(2,1)
H

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.]
 [ 0. -1. -1.  0.]
 [ 0. -1. -1.  0.]
 [ 0.  0.  0. -2.]]

In [173]:
H.eigenstates()

(array([-2., -2.,  0.,  0.]),
 array([Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [0.]
  [1.]],
        Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.        ]
  [0.70710678]
  [0.70710678]
  [0.        ]],
        Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[ 0.        ]
  [-0.70710678]
  [ 0.70710678]
  [ 0.        ]],
        Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[1.]
  [0.]
  [0.]
  [0.]]], dtype=object))